# Задание 1


### Задачи для Практикума по программированию. Общая тема задания «Шахматный симулятор: объектно-ориентированная версия».

### Реализовать программу, которая позволяет играть в шахматы на компьютере. Взаимодействие с программой производится через консоль (базовый вариант). Игровое поле изображается в виде 8 текстовых строк, плюс строки с буквенным обозначением столбцов (см. пример на Рис. 1) и перерисовывается при каждом изменении состояния поля. При запросе данных от пользователя программа сообщает, что ожидает от пользователя (например, позицию фигуры для следующего хода белыми; целевую позицию выбранной фигуры) и проверяет корректность ввода (допускаются только ходы соответствующие правилам шахмат; поддержка рокировки, сложных правил для пешек и проверки мата вынесена в отдельные пункты). Программа должна считать количество сделанных ходов.

In [25]:
class Piece:
    """Базовый класс для шахматных фигур, содержит общую логику перемещений."""

    def __init__(self, color, symbol):
        """
        Инициализирует фигуру с заданным цветом и символом.

        Args:
            color (str): Цвет фигуры ('white' или 'black')
            symbol (str): Символ для отображения на доске
        """
        self.color = color
        self.symbol = symbol

    def __str__(self):
        """Возвращает символьное представление фигуры."""
        return self.symbol

    def get_piece_color(self):
        """Возвращает цвет фигуры."""
        return self.color if self else None

    def moves_r_b_q(self, board, start, directions):
        """
        Рассчитываем ходы для фигур с линейным перемещением (ладья, слон, ферзь).

        Args:
            board: Шахматная доска
            start (tuple): Начальная позиция (x, y)
            directions (list): Список направлений движения

        Returns:
            list: Список допустимых позиций
        """
        start_x, start_y = start
        moves = []

        for dir_x, dir_y in directions:
            current_x, current_y = (start_x + dir_x, start_y + dir_y)
            while board.is_valid_position((current_x, current_y)):
                if board.get_piece((current_x, current_y)):
                    if board.get_piece((current_x, current_y)).color == self.color:
                        moves.append((current_x, current_y))
                        break
                    break
                moves.append((current_x, current_y))
                current_x += dir_x
                current_y += dir_y
        return moves

    def moves_n_k(self, board, start, directions):
        """
        Рассчитывает ходы для фигур с точечным перемещением (конь, король).

        Args:
            board: Шахматная доска
            start (tuple): Начальная позиция (x, y)
            directions (list): Список направлений движения

        Returns:
            list: Список допустимых позиций
        """
        start_x, start_y = start
        moves = []

        for dir_x, dir_y in directions:
            current_pos = (start_x + dir_x, start_y + dir_y)
            if board.is_valid_position(current_pos):
                if board.get_piece(current_pos):
                    if board.get_piece(current_pos).color != self.color:
                        moves.append(current_pos)
                        continue
                    continue
                moves.append(current_pos)
        return moves



class Pawn(Piece):
    def __init__(self, color):
        super().__init__(color, 'P' if color == 'white' else 'p')

    def get_possible_moves(self, board, start):
        """
        Рассчитывает возможные ходы пешки, включая взятие и начальный ход на 2 клетки.

        Args:
            board: Шахматная доска
            start (tuple): Текущая позиция (x, y)

        Returns:
            list: Список допустимых позиций
        """
        direction = -1 if self.color == 'white' else 1
        start_x, start_y = start
        moves = []

        # Базовое перемещение вперед
        if board.is_valid_position((start_x + direction, start_y)) and not board.get_piece((start_x + direction, start_y)):
            moves.append((start_x + direction, start_y))

        # Начальный ход на 2 клетки
        if (self.color == 'white' and start_x == 6) or (self.color == 'black' and start_x == 1):
            if board.is_valid_position((start_x + direction * 2, start_y)) and not board.get_piece((start_x + direction * 2, start_y)):
                moves.append((start_x + direction * 2, start_y))

        #pawn_kill (Пешка кушает другую фигуру)
        for pawn_kill in [-1, 1]:
            kill_pos = (start_x + direction, start_y + pawn_kill)
            if board.is_valid_position(kill_pos):
                piece = board.get_piece(kill_pos)
                if piece and piece.color != self.color:
                    moves.append(kill_pos)
        return moves



class Rook(Piece):
    def __init__(self, color):
        super().__init__(color, 'R' if color == 'white' else 'r')
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    def get_possible_moves(self, board, start):
        return self.moves_r_b_q(board, start, self.directions)



class Bishop(Piece):
    def __init__(self, color):
        super().__init__(color, 'B' if color == 'white' else 'b')
        self.directions = [(-1, -1), (1, 1), (-1, 1), (1, -1)]

    def get_possible_moves(self, board, start):
        return self.moves_r_b_q(board, start, self.directions)



class Queen(Piece):
    def __init__(self, color):
        super().__init__(color, 'Q' if color == 'white' else 'q')
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]

    def get_possible_moves(self, board, start):
        return self.moves_r_b_q(board, start, self.directions)



class Knight(Piece):
    def __init__(self, color):
        super().__init__(color, 'N' if color == 'white' else 'n')
        self.directions = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]

    def get_possible_moves(self, board, start):
        return self.moves_n_k(board, start, self.directions)



class King(Piece):
    def __init__(self, color):
        super().__init__(color, 'K' if color == 'white' else 'k')
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]

    def get_possible_moves(self, board, start):
        return self.moves_n_k(board, start, self.directions)

In [27]:
class Board:
    def __init__(self):
        self.board = [[None for _ in range(8)] for _ in range(8)]
        self.setup_board()

    def setup_board(self):
        for i in range(8):
            self.board[1][i], self.board[6][i] = Pawn('black'), Pawn('white')

        self.board[0][0], self.board[0][7] = Rook('black'), Rook('black')
        self.board[0][1], self.board[0][6] = Knight('black'), Knight('black')
        self.board[0][2], self.board[0][5] = Bishop('black'), Bishop('black')
        self.board[0][3], self.board[0][4] = Queen('black'), King('black')
        self.board[2][0], self.board[2][0],


        self.board[7][0], self.board[7][7] = Rook('white'), Rook('white')
        self.board[7][1], self.board[7][6] = Knight('white'), Knight('white')
        self.board[7][2], self.board[7][5] = Bishop('white'), Bishop('white')
        self.board[7][3], self.board[7][4] = Queen('white'), King('white')
        print()

    def display(self):
        """
        Отображение шахматной доски
        Просто дорисовывает цифры и буквы по краям, доска остаётся такой же
        """
        print("   A B C D E F G H")
        print(" +-----------------+")
        for index, row in enumerate(self.board):
            print(f"{8 - index}| {' '.join([str(piece) if piece else '.' for piece in row])} |{8 - index}")
        print(" +-----------------+")
        print("   A B C D E F G H")

    def get_piece(self, position):
        """
        Получение фигуры по позиции на доске для проверки корректности выбора фигуры
        :param position: Позиция
        :return: Фигура на указанной позиции
        """
        # x, y = self.position_to_indices(position)
        # return self.board[x][y]

        string, col = position
        return self.board[string][col] if self.is_valid_position(position) else None


    def move_piece(self, start, end):
        """
        Перемещение фигуры с одной позиции на другую
        :param start: Начальная позиция
        :param end: Конечная позиция
        """
        start_x, start_y = start
        end_x, end_y = end

        piece = self.board[start_x][start_y]
        self.board[end_x][end_y] = piece
        self.board[start_x][start_y] = None

    @staticmethod
    def is_valid_position(position):
        """Метод для проверки корректности введенных координат(не выходят ли они за пределы поля).

        Args:
            position (tuple): координаты позиции

        Returns:
            bool: истина если координаты валидны и ложь в противном случае
        """

        string, col = position
        return 0 <= string < 8 and 0 <= col < 8

    @staticmethod
    def position_to_indices(position):
        """
        Преобразование шахматной позиции в индексы массива
        :param position: Позиция в формате 'e4'
        :return: Кортеж (x, y) — индексы строки и столбца в формате (4, 4) - координата 'e4',
                 но в виде индексов вложенного списка
        """
        file = position[0].lower()
        rank = int(position[1])
        x = 8 - rank
        y = ord(file) - ord('a')
        return x, y

# Пример использования
if __name__ == "__main__":
    board = Board()
    board.display()


   A B C D E F G H
 +-----------------+
8| r n b q k b n r |8
7| p p p p p p p p |7
6| . . . . . . . . |6
5| . . . . . . . . |5
4| . . . . . . . . |4
3| . . . . . . . . |3
2| P P P P P P P P |2
1| R N B Q K B N R |1
 +-----------------+
   A B C D E F G H


In [ ]:
class Game:
    def __init__(self):
        self.board = Board()
        self.current_turn = 'white'
        self.move_count = 0
        self.move_history = []

    def hints(self, start):
        """Метод для подсказки куда можно сходить и какие фигуры можно съесть.

        Args:
            start (tuple): координаты откуда сходить
        """

        if self.board.get_piece(start):
            current_piece = self.board.get_piece(start)

            print('\n   A B C D E F G H')
            print(' +-----------------+')
            for index0, row in enumerate(self.board.board):
                print(f'{8 - index0}|', end=' ')
                for index1, piece in enumerate(row):
                    if (index0, index1) in current_piece.get_possible_moves(self.board, start):
                        print(f"\x1B[1;41m{self.board.get_piece((index0, index1)) if piece else '.'}\x1B[0m", end=' ')
                    else:
                        print(self.board.get_piece((index0, index1)) if piece else '.', end=' ')
                print(f'|{8 - index0}')
            print(' +-----------------+')
            print('   A B C D E F G H')
        else:
            return None

    def start(self):
        while True:
            self.board.display()
            print(f"Ход {'белых' if self.current_turn == 'white' else 'чёрных'}.")

            start = input('Введите координаты целевой фигуры: ').strip()
            if not start:
                continue

            if start == 'undo':
                if not self.move_history:
                    print("Нет ходов для отмены.")
                    continue
                move_count, start, end, current_turn = self.move_history.pop()
                print(f"Ход {'белых' if current_turn == 'white' else 'чёрных'}.")
                index_start = self.board.position_to_indices(start)
                self.hints(index_start)
                index_end = self.board.position_to_indices(end)
                self.board.move_piece(index_end, index_start)
                self.move_count -= 1
                self.current_turn = 'black' if self.current_turn == 'white' else 'white'
                continue


            if len(start) != 2 or start[0] not in 'abcdefgh' or start[1] not in '12345678':  # Проверка на корректный ввод
                print("Некорректный ввод. Введите координаты в формате 'e2'.\n")
                continue

            index_start = self.board.position_to_indices(start)

            piece = self.board.get_piece(index_start)
            if not piece:
                print('На этой позиции нет фигуры.')
                continue

            print('Выбранная фигура: ', piece)

            if piece.color != self.current_turn:
                print('Вы не можете ходить чужой фигурой.')
                continue

            self.hints(index_start)

            end = input('Введите целевые координаты хода: ').strip()
            if not end:
                continue

            if len(end) != 2 or end[0] not in 'abcdefgh' or end[1] not in '12345678':  # Проверка на корректный ввод
                print("Некорректный ввод. Введите координаты в формате 'e4'.\n")
                continue

            index_end = self.board.position_to_indices(end)

            if index_end not in piece.get_possible_moves(self.board, index_start):
                print('Недопустимый ход.')
                continue

            self.board.move_piece(index_start, index_end)
            self.move_count += 1
            self.move_history.append([self.move_count, start, end, self.current_turn])
            self.current_turn = 'black' if self.current_turn == 'white' else 'white'



if __name__ == '__main__':
    game = Game()
    game.start()


   A B C D E F G H
 +-----------------+
8| r n b q k b n r |8
7| p p p p p p p p |7
6| . . . . . . . . |6
5| . . . . . . . . |5
4| . . . . . . . . |4
3| . . . . . . . . |3
2| P P P P P P P P |2
1| R N B Q K B N R |1
 +-----------------+
   A B C D E F G H
Ход белых.
Введите координаты целевой фигуры: a2
Выбранная фигура:  P

   A B C D E F G H
 +-----------------+
8| r n b q k b n r |8
7| p p p p p p p p |7
6| . . . . . . . . |6
5| . . . . . . . . |5
4| . . . . . . . . |4
3| . . . . . . . . |3
2| P P P P P P P P |2
1| R N B Q K B N R |1
 +-----------------+
   A B C D E F G H
